# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [47]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [9]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [31]:
import numpy as np

w = data[data.race=='w']
b = data[data.race=='b']

# Q1 

print("Callback Rate for whites and blacks are",np.mean(w.call)
      ,"and",np.mean(b.call))

# This preliminary analysis shows a substantial difference in average callback rates. Now, we will use the bootstrap
# hypothesis testing to check whether the difference is statistical difference. We will set alpha at 0.05

# A two-sample sample proportion hypthesis test is required to explore whether there is a significant difference
# 

# Q2 Null Hypothesis: There is no significant difference between black and white races in the callback rate 

# Alternative Hypothesis: There is a significant difference between races in callback rate. This can be either
# that white people have a higher callback rate than black people or vice versa

Callback Rate for whites and blacks are 0.09650924056768417 and 0.0644763857126236


In [52]:
# Your solution to Q3 here

## bootstrap

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty((size))

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def diff_of_means(data1,data2):
    return np.abs(np.mean(data1) - np.mean(data2))


diff_observed = diff_of_means(w.call,b.call)

perm_replicates = draw_perm_reps(w.call, b.call,
                                 diff_of_means, size=10000)

p = np.sum(perm_replicates >= diff_observed) / len(perm_replicates)
# print(p)



In [51]:
## In our bootstrap method, out of 10,000 trials, we did not find a single trial in which permutation of the two
# arrays resulted in a difference in callback rates as extreme as the one we observed in the sample. Thus, we can
# conclude that there is definitely a statistical significant difference in callback rates

In [48]:
## frequentist

total_p = np.mean(data.call)
z_score = (np.mean(w.call) - np.mean(b.call))/(total_p*(1-total_p)/len(data))**0.5

p_val = stats.norm.sf(np.abs(z_score))*2

## this p_val of 2E-16 shows that there is almost no possibility of having such a difference in proportions of resumes
# given a callback if we assume the null hypothesis. Thus, we can reject this null hypothesis and can state that
# there is a significant differnce in proportions of callbacks between resumes with white and black sounding names.


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [53]:
## While we have definitely shown that race is a relevant indicator of callback rates, this could be due to a 
# correlation rather than a causation. To definitely prove such a claim, we must examine the w and b data sets 
# and make sure other variables are evenly distributed between the two samples. Since the samples were described
# as having been formed randomly, it is unlikely that any other variable might be able to explain the difference,
# but it should be explored nontheless. 

# We have proved that race is a factor, but not the most important factor. There might be another categorization
# or continuous variable that results in a higher impact on the resulting callback rate. 
